# Cresi Output Example
Cresi, which stands for City-scale Road Extraction from Satellite Imagery, is a library, well, for extracting road segments from satellite imagery. This library includes tools for segmenting a raster satellite image along with the corresponding vector labels. It also contains all the necessary tools to deploy already trained models that have been trained on the Spacenet 6 dataset. The output of the models are geo-referenced NetworkX graph for the corresponding road segments found in the input raster image. The model is also able to predict the speed of the roads. Furthermore, given 2 points on the output vector the model is able to optimize for distance or time, given that it can predict the speed of the segmented roads.

In this notebook I will be visualizing some of the outputs that I was able to generate on an input image of the Bekaa Valley in Lebanon. 

## Import Libraries

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
import rasterio
from pathlib import Path
import shapely
import json
import gdal
print('import complete')

import complete


## Setup Required Paths

In [2]:
# load directories
data_path = Path(Path.cwd()/'data')
cresi_path = Path(data_path/'cresi')

# load paths to files
sample_image_path = Path(cresi_path/'Ortho_Sample.tif')
vectors_path = Path(cresi_path/'wkt_vectors.csv')


## Reprojecting Raster Image CRS to 4326

In [3]:
with rasterio.open(sample_image_path) as r:
    print('Number of Bands: ',r.count,'\n')
    print('Bounds of the Raster File: ',r.bounds,'\n')
    print('Raster File CRS: ',r.crs,'\n')


Number of Bands:  3 

Bounds of the Raster File:  BoundingBox(left=767588.6493501049, bottom=3745779.3534929864, right=771234.6493501049, top=3748007.3534929864) 

Raster File CRS:  EPSG:32636 



In [7]:
def reproject_raster_gdal(in_path:str,epsg:str,out_path:str=None):
    in_path = str(in_path)
    if not out_path:
        out_path = in_path[:-4]+epsg+'.tif'
    input_raster = gdal.Open(in_path)
    gdal.Warp(out_path,input_raster,dstSRS=epsg)
    return out_path

In [8]:
out_path = Path(reproject_raster_gdal(in_path=sample_image_path,epsg='EPSG:4326'))

In [9]:
sample_transform_path = Path(cresi_path/'sample_4326.tif')

In [10]:
with rasterio.open(out_path) as r:
    print('Number of Bands: ',r.count,'\n')
    print('Bounds of the Raster File: ',r.bounds,'\n')
    print('Raster File CRS: ',r.crs,'\n')

Number of Bands:  3 

Bounds of the Raster File:  BoundingBox(left=35.891007792358295, bottom=33.817464210009774, right=35.93103767327084, top=33.83846264766881) 

Raster File CRS:  EPSG:4326 



## Visualizing input raster file

In [11]:
def get_bbox(chip_path):
    with rasterio.open(chip_path) as c:
        bounds = c.bounds
    # [[top_left],[bottom_right]]
    bbox = [[bounds[3],bounds[0]],[bounds[1],bounds[2]]]
    return bbox

In [12]:
def read_raster(chip_path):
    with rasterio.open(chip_path) as c:
        return c.read()

In [13]:
def rasterio_affine_to_shapely_affine(matrix):
    return [matrix[0],matrix[1],matrix[3],matrix[4],matrix[2],matrix[5]]

### Create Folium Map Object

In [14]:
raster_bbox = get_bbox(sample_transform_path)

In [15]:
map = folium.Map(
    location=raster_bbox[0],
    zoom_start=18)

### Convert raster to image
In order to plot raster image it needs to be converted from raster format (bands, rows, columns) to (rows, columns, bands) this can be done in rasterio using the functions below

In [16]:
from rasterio.plot import reshape_as_raster, reshape_as_image

In [17]:
raster = read_raster(out_path)
raster.shape

(3, 4078, 7774)

In [18]:
image = reshape_as_image(raster)
image.shape

(4078, 7774, 3)

### Add raster layer on top base folium  map

In [19]:
folium.raster_layers.ImageOverlay(
    image=image,
    bounds=raster_bbox,
    opacity=0.6,
    name='raster'   
).add_to(map);

### Read output Cresi CSV

In [20]:
df = pd.read_csv(vectors_path)

### Convert Well Known Text Format to Geopandas Geometry

In [21]:
df['geometry'] = df['WKT_Pix'].apply(shapely.wkt.loads)

In [22]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [23]:
gdf.head()

,ImageId,WKT_Pix,geometry
0,Ortho_Sample_clip,"LINESTRING (1352.0 2.0, 1347.0 93.0, 1339.0 14...","LINESTRING (1352.000 2.000, 1347.000 93.000, 1..."
1,Ortho_Sample_clip,"LINESTRING (2547.0 2.0, 2574.0 89.0, 2586.0 14...","LINESTRING (2547.000 2.000, 2574.000 89.000, 2..."
2,Ortho_Sample_clip,"LINESTRING (3585.0 2.0, 3612.0 20.0, 3639.0 30...","LINESTRING (3585.000 2.000, 3612.000 20.000, 3..."
3,Ortho_Sample_clip,"LINESTRING (4025.0 2.0, 4037.0 21.0, 4069.0 62...","LINESTRING (4025.000 2.000, 4037.000 21.000, 4..."
4,Ortho_Sample_clip,"LINESTRING (4786.0 2.0, 4831.0 47.0, 4835.0 58.0)","LINESTRING (4786.000 2.000, 4831.000 47.000, 4..."


### Set Original CRS

In [25]:
with rasterio.open(str(sample_image_path)) as r:
    crs = r.crs
    m = r.transform
    print(crs)
    print(m)


EPSG:32636
| 0.50, 0.00, 767588.65|
| 0.00,-0.50, 3748007.35|
| 0.00, 0.00, 1.00|


In [24]:
gdf.crs = crs
gdf.crs

'EPSG:32636'

### Use original image affine transformation information to convert pixel coordinates geocoordinates

In [26]:
# get affine matrix into format accepted by shapely and geopandas
matrix = rasterio_affine_to_shapely_affine(matrix = m)
matrix

[0.5, 0.0, 0.0, -0.5, 767588.6493501049, 3748007.3534929864]

In [27]:
gdf['geometry'] = gdf['geometry'].affine_transform(matrix)
gdf.head()

,ImageId,WKT_Pix,geometry
0,Ortho_Sample_clip,"LINESTRING (1352.0 2.0, 1347.0 93.0, 1339.0 14...","LINESTRING (768264.649 3748006.353, 768262.149..."
1,Ortho_Sample_clip,"LINESTRING (2547.0 2.0, 2574.0 89.0, 2586.0 14...","LINESTRING (768862.149 3748006.353, 768875.649..."
2,Ortho_Sample_clip,"LINESTRING (3585.0 2.0, 3612.0 20.0, 3639.0 30...","LINESTRING (769381.149 3748006.353, 769394.649..."
3,Ortho_Sample_clip,"LINESTRING (4025.0 2.0, 4037.0 21.0, 4069.0 62...","LINESTRING (769601.149 3748006.353, 769607.149..."
4,Ortho_Sample_clip,"LINESTRING (4786.0 2.0, 4831.0 47.0, 4835.0 58.0)","LINESTRING (769981.649 3748006.353, 770004.149..."


### Convert to polygons to CRS ESPG:4326

In [28]:
gdf['geometry'] = gdf['geometry'].to_crs(epsg=4326)
gdf.head()

,ImageId,WKT_Pix,geometry
0,Ortho_Sample_clip,"LINESTRING (1352.0 2.0, 1347.0 93.0, 1339.0 14...","LINESTRING (35.89898 33.83828, 35.89894 33.837..."
1,Ortho_Sample_clip,"LINESTRING (2547.0 2.0, 2574.0 89.0, 2586.0 14...","LINESTRING (35.90543 33.83813, 35.90556 33.837..."
2,Ortho_Sample_clip,"LINESTRING (3585.0 2.0, 3612.0 20.0, 3639.0 30...","LINESTRING (35.91103 33.83800, 35.91118 33.837..."
3,Ortho_Sample_clip,"LINESTRING (4025.0 2.0, 4037.0 21.0, 4069.0 62...","LINESTRING (35.91341 33.83794, 35.91347 33.837..."
4,Ortho_Sample_clip,"LINESTRING (4786.0 2.0, 4831.0 47.0, 4835.0 58.0)","LINESTRING (35.91751 33.83784, 35.91775 33.837..."


In [29]:
gdf.crs = 'EPSG:4326'

In [31]:
gdf.crs

'EPSG:4326'

In [32]:
gdf = gdf.drop('WKT_Pix',axis=1)

In [33]:
gdf.head()

,ImageId,geometry
0,Ortho_Sample_clip,"LINESTRING (35.89898 33.83828, 35.89894 33.837..."
1,Ortho_Sample_clip,"LINESTRING (35.90543 33.83813, 35.90556 33.837..."
2,Ortho_Sample_clip,"LINESTRING (35.91103 33.83800, 35.91118 33.837..."
3,Ortho_Sample_clip,"LINESTRING (35.91341 33.83794, 35.91347 33.837..."
4,Ortho_Sample_clip,"LINESTRING (35.91751 33.83784, 35.91775 33.837..."


In [34]:
print(gdf['geometry'][0])

LINESTRING (35.8989806188425 33.83828214754081, 35.89893978357955 33.83787293771328, 35.89888899716685 33.83764876420994, 35.89883584687392 33.83719477035845, 35.89885826684362 33.83705902167562, 35.89899303041007 33.83673131987481, 35.89904578346533 33.83669401955837)


In [35]:
raster_bbox

[[33.83846264766881, 35.891007792358295],
 [33.817464210009774, 35.93103767327084]]

### Save GeoDataFrame as GeoJSON File

In [36]:
gdf.to_file(Path(cresi_path/"geojson.geojson"), driver="GeoJSON")

### Add vector layers on top of base folium map

In [37]:
def geojson_to_json(geom_path):
    with open(geom_path,'r') as g:
        geojson = g.read()
    return json.loads(geojson)

In [38]:
geojson_path = Path(cresi_path/"geojson.geojson")

In [39]:
folium.GeoJson(geojson_to_json(geojson_path),name='vector').add_to(map);

### Add Layer control to toggle between the different layers

In [40]:
folium.LayerControl().add_to(map);

In [41]:
map.save(str(Path(cresi_path/'map.html')))

In [42]:
map